In [ ]:
!pip3 install requests
!pip3 install pyodbc
import json, requests, itertools, os, pandas as pd, numpy as np, pyodbc
from urllib.parse import urljoin

In [ ]:
base_url = 'https://siu.admin.12twenty.com/api'
auth_key = 'ig5eb2bqnq5cec1hrud8y'
auth_url = base_url + '/client/generateAuthenticationToken?key=' + auth_key

bearer_token = requests.get(auth_url).text
auth_header = {'Authorization': 'Bearer ' + bearer_token}

In [ ]:
students_url = base_url + '/V2/students'
students_params = {'PageSize' : '500'}
twelve_twenty_students_json = requests.get(students_url, headers=auth_header, params=students_params)
#print(json.dumps(studentsJson.json(), indent=2))

In [ ]:
twelve_twenty_students_json = json.loads(twelve_twenty_students_json.text)
twelve_twenty_students_page = pd.json_normalize(twelve_twenty_students_json['Items'])
twelve_twenty_students_page.head()

In [ ]:
twelve_twenty_students = []
twelve_twenty_students.append(twelve_twenty_students_page)
num_pages = twelve_twenty_students_json['NumberOfPages']
for page in range(2, num_pages + 1):
    students_params = {'PageSize' : '500', 'PageNumber': page}
    twelve_twenty_students_json = json.loads(requests.get(students_url, headers=auth_header, params=students_params).text)
    twelve_twenty_students_page = pd.json_normalize(twelve_twenty_students_json['Items'])
    twelve_twenty_students.append(twelve_twenty_students_page)

In [ ]:
twelve_twenty_students = pd.concat(twelve_twenty_students)
print(twelve_twenty_students.count())
twelve_twenty_students

In [ ]:
print(twelve_twenty_students.columns.tolist())

columnMapping = {'Id':'12TwentyId',
                 'FirstName':'FirstName',
                 'MiddleName':'MiddleName',
                 'LastName' : 'LastName',
                 'EmailAddress' : 'EmailAddress',
                 'IsEnrolled': 'IsEnrolled',
                 'StudentId': 'StudentId',
                 'College.Name': 'College',
                 'Program.Name': 'Program',
                 'GraduationYearId' : 'GraduationYear',
                 'GraduationTerm' : 'GraduationTerm',
                 'DegreeLevel.Name' : 'DegreeLevel',
                 'CustomAttributeValues.custom_attribute_10888805132042': 'AppliedForGraduation'
}

for column in twelve_twenty_students.columns:
    if column not in columnMapping.keys():
        twelve_twenty_students.drop(column, axis=1, inplace=True)

twelve_twenty_students.rename(columns = columnMapping, inplace='True')

twelve_twenty_students.set_index('StudentId')
twelve_twenty_students


In [ ]:
sql_connection = pyodbc.connect("Driver={SQL Server};"
                      "Server=itapp-ssis-dev;"
                      "Database=MableySandbox;"
                      "Trusted_Connection=yes;")


In [ ]:
ods_query = "SELECT * FROM TwelveTwentyStudents;"
ods_students = pd.read_sql(ods_query,sql_connection)
ods_students.insert(len(ods_students.columns), 'ActionNeeded', 'None')
ods_students.insert(len(ods_students.columns), 'Result', '')
ods_students.insert(len(ods_students.columns), 'Message', '')
#strip year from graduation term
ods_students['GraduationTerm'] = ods_students['GraduationTerm'].str.replace('\d*', '', regex=True)
ods_students.set_index('StudentId')

In [ ]:
ods_students.loc[~ods_students['StudentId'].isin(twelve_twenty_students['StudentId']), 'ActionNeeded'] = 'Insert'
ods_students.loc[~ods_students['StudentId'].isin(twelve_twenty_students['StudentId'])]

In [ ]:
# idx = odsStudentsDataFrame.index.intersection(allStudentData)
# odsStudentsDataFrame.loc[idx].eq(allStudentData.loc[idx])
# studentsToUpdate = odsStudentsDataFrame.where(odsStudentsDataFrame['StudentId'] == allStudentData['StudentId'])

merged_students = pd.merge(ods_students, twelve_twenty_students, on ='StudentId', how='left', suffixes=['_ODS', '_12Twenty'])
merged_students

In [ ]:
merged_students.loc[merged_students['EmailAddress_ODS'] != (merged_students['EmailAddress_12Twenty'])] = 'Update'

# merged_students['EmailAddress_ODS'].equals(merged_students['EmailAddress_12Twenty']),'Update', merged_students['ActionNeeded']))
                                            # &
                                            # (merged_students['FirstName_ODS']!= merged_students['FirstName_12Twenty']) &
                                            # (merged_students['LastName_ODS']!= merged_students['LastName_12Twenty']) &
                                            # (merged_students['DegreeLevel_ODS']!= merged_students['DegreeLevel_12Twenty']) &
                                            # (merged_students['GraduationYear_ODS']!= merged_students['GraduationYear_12Twenty']) &
                                            # (merged_students['GraduationTerm_ODS']!= merged_students['GraduationTerm_12Twenty']) &
                                            # (merged_students['AppliedForGrad_ODS' != merged_students['AppliedForGraduation']]) &
                                            # (merged_students['IsEnrolled_ODS' != merged_students['IsEnrolled_12Twenty']]),
                             
merged_students